In [419]:
# import packages

from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
import numpy as np

# import data and assign a rank to certain metrics

resorts = (
    pd.read_csv("data/resorts.csv", encoding = "ISO-8859-1")
    .assign(
        country_elevation_rank = lambda x: x.groupby("Country", as_index=False)["Highest point"].rank(ascending=False),
        country_price_rank = lambda x: x.groupby("Country", as_index=False)["Price"].rank(ascending=False),
        country_slope_rank = lambda x: x.groupby("Country", as_index=False)["Total slopes"].rank(ascending=False),
        country_lift_rank = lambda x: x.groupby("Country", as_index=False)["Total lifts"].rank(ascending=False),
        global_elevation_rank = lambda x: x["Highest point"].rank(ascending=False),
        global_price_rank = lambda x: x["Price"].rank(ascending=False),
        global_slope_rank = lambda x: x["Total slopes"].rank(ascending=False),
        global_lift_rank = lambda x: x["Total lifts"].rank(ascending=False),
    ))

resorts["Resorts_short"] = resorts["Resort"].str.split("-", n=1).str[0]
resorts["Resorts_short"] = resorts["Resorts_short"].str.split("(", n=1).str[0]

# application

app = Dash(__name__, external_stylesheets = [dbc.themes.BOOTSTRAP])

load_figure_template("BOOTSTRAP")

app_style = {"backgroundColor": "#F5F7FB",
             "color": "#33394C",
            "text-align": "left",
            "height": "100vh"}

app.layout = (
html.Div(
    style = app_style,
    children = [
        dbc.NavbarSimple(
                    children=[dbc.NavItem(dbc.NavLink("GitHub", href="https://github.com/kieswetter-hub/Ski-resort-dashboard"))],
                            brand="Exploring Global Ski Resorts",
                            color="#4E7CFF",
                            dark=True),
        dbc.Tabs(id = "tabs",
        children = [
            dbc.Tab(label = "Resort Map",
                    tab_style={"margin": 0,
                               "width": "50%",
                               "font-size": "18px",
                               "backgroundColor": "#F5F7FB"},
                    label_style={"text-align": "center",
                                 "color": "#33394C"},
                    active_label_style={"background-color": "#F5F7FB",
                                        "color": "black"},
                            children = [
                                dbc.Container([
                                    dbc.Row([
                                        html.P(),
                                        html.H2(id="title", style = {"text-align": "center"}),
                                        html.P()]),
                                    dbc.Row([dbc.Col([
                                                      dbc.Card(style = {"text-align":"left",
                                                                        "backgroundColor": "#fefefe",
                                                                        "padding" : "20px"},
                                                        children = [
                                                            html.P("Select a price", style = {"font-size": 18}),
                                                            dcc.Slider(id = "maxprice",
                                                                       min = 0,
                                                                       max = 150,
                                                                       step = 25,
                                                                       value = 100,
                                                                       marks={i:{"label": f'${i}', 
                                                                                 "style": {"fontSize": 14}} 
                                                                                 for i in range(0, 151, 25)}),
                                                            html.Br(),
                                                            html.P("Select features", style = {"font-size": 18}),
                                                            dcc.Checklist(id = "selectFeatures",
                                                                          options = [
                                                                              {'label': '   Summer skiing', 'value': 'Summer skiing'},
                                                                              {'label': '   Night skiing', 'value': 'Nightskiing'},
                                                                              {'label': '   Snow parks', 'value': 'Snowparks'}],
                                                                              value = [])])], width = 4),
                                            dbc.Col([
                                                      dbc.Card(style = {"text-align":"center",
                                                                        "backgroundColor": "#fefefe"},
                                                        children = [
                                                            dcc.Graph(id = "map")])], width = 8)
                                            ]) # closes row
                                ]) # closes container
                        ]), # closes children tab 1                       
            dbc.Tab(label = "Country Profile",
                    tab_style={"margin": 0,
                               "width": "50%",
                               "font-size": "18px",
                               "backgroundColor": "#F5F7FB"},
                    label_style={"text-align": "center",
                                 "color": "#33394C"},
                    active_label_style={"background-color": "#F5F7FB",
                                        "color": "black"},
                            children = [
                                dbc.Container([
                                    dbc.Row([
                                        html.P(),
                                        html.H2(id="title2", style = {"text-align": "center"}),
                                        html.P()
                                        ]),
                                    dbc.Row([dbc.Col([
                                                      dbc.Card(style = {"text-align":"left",
                                                                        "backgroundColor": "#fefefe",
                                                                        "padding" : "20px"},
                                                        children = [
                                                                html.P("Select a continent", style = {"font-size": 18}),
                                                                dcc.Dropdown(id = "selectContinent",
                                                                             options =[{'label': c, 'value': c} for c in np.sort(resorts["Continent"].unique())],
                                                                             value = "Europe"),
                                                                html.Br(),
                                                                html.P("Select a country", style = {"font-size": 18}),
                                                                dcc.Dropdown(id="selectCountry",
                                                                             value = "Norway"),
                                                                html.Br(),
                                                                html.P("Select a metric", style = {"font-size": 18}),
                                                                dcc.Dropdown(id = "selectMetric",
                                                                             options = ["Price","Highest point","Total slopes","Total lifts"],
                                                                             value = "Price"),
                                                                html.Br(),                                                                                                                                                       
                                                                    ])], 
                                                                    width = 3),
                                            dbc.Col([
                                                      dbc.Card(style = {"text-align":"center",
                                                                        "backgroundColor": "#fefefe"},
                                                        children = [
                                                            dcc.Graph(id = "barchart", hoverData = {"points": [{"customdata": [""]}]})])], 
                                                            width = 7),
                                             dbc.Col([dbc.Card(
                                                 children = [
                                                 dbc.CardHeader("Local Rank (Global)"),
                                                 dbc.CardBody([
                                                     html.H5(id = "resort-name", style = {"padding-bottom": "10px"}),
                                                     html.P(id = "price-rank"),
                                                     html.P(id = "elevation-rank"),
                                                     html.P(id = "slope-rank"),
                                                     html.P(id = "lift-rank"),
                                                            ]),
                                                            ]
                                                      )], 
                                                      width = 2)
                                            ])])
                                ])
                    ])
        ])
)

@app.callback(
    Output("title", "children"),
    Output("map", "figure"),
    Input("maxprice", "value"),
    Input("selectFeatures", "value"),
)

def update_map(pricelimit, features):
    
    df_filtered = resorts.query("Price <= @pricelimit")

    title = f"Slope density of ski resorts under ${pricelimit}"

    if len(features) == 1:
        if "Nightskiing" in features:
            df_filtered = df_filtered.query('Nightskiing == "Yes"')
        elif "Summer skiing" in features:
            df_filtered = df_filtered.query('`Summer skiing` == "Yes"')
        else:
            df_filtered = df_filtered.query('Snowparks == "Yes"')
    elif len(features) == 2:
        if "Nightskiing" in features and "Summer skiing" in features :
            df_filtered = df_filtered.query("`Summer skiing` == 'Yes' and Nightskiing == 'Yes'")
        if "Nightskiing" in features and "Snowparks" in features :
            df_filtered = df_filtered.query("Snowparks == 'Yes' and Nightskiing == 'Yes'")
        if "Summer skiing" in features and "Snowparks" in features :
            df_filtered = df_filtered.query("`Summer skiing` == 'Yes' and Snowparks == 'Yes'")
    elif len(features) == 3:
            df_filtered = df_filtered.query("`Summer skiing` == 'Yes' and Nightskiing == 'Yes' and Snowparks == 'Yes'")
    else:
        df_filtered = df_filtered

    figure = (px.density_mapbox(
        df_filtered,
        lat = "Latitude",
        lon = "Longitude",
        z = "Total slopes",
        mapbox_style = "carto-positron",
        zoom = 2.5,
        center = {"lat": 52.5, "lon": 15},
        radius = 15,
        height = 525,
        hover_name="Resort",
        hover_data={'Latitude':False, 'Longitude':False})
        .update_layout(
            geo_bgcolor = "#F5F7FB",
            paper_bgcolor = "#fefefe"))
    
    return title, figure

@app.callback(
        Output("selectCountry", "options"),
        Input("selectContinent", "value"),
)

def populateCountryDropDown(continent):  
    if not continent:
        raise PreventUpdate  
    
    country_list = np.sort(resorts.query("Continent == @continent")["Country"].unique())

    return country_list


@app.callback(
    Output("title2", "children"),
    Output("barchart", "figure"),
    Input("selectCountry", "value"),
    Input("selectMetric", "value")
)

def update_bar(country, metric):
    if not country:
        raise PreventUpdate

    df_bar = resorts.query("Country == @country").sort_values(by = metric, ascending = False).head(10)
    df_bar
    
    title2 = f"Top resorts in {country} based on {metric.lower()}"

    figure2 = (px.bar(df_bar,
                      x = "Resorts_short",
                      y = metric,
                      labels = {"Resorts_short": "Resort"},
                      hover_name="Resort",
                      hover_data={'Resort':True},
                      custom_data=["Resort", "country_price_rank"])
                      .update_traces(
                          showlegend=False,
                          marker_color="#4E7CFF",
                          opacity = 0.85))
    
    return title2, figure2

@app.callback(
    Output("resort-name", "children"),
    Output("price-rank", "children"),
    Output("elevation-rank", "children"),
    Output("slope-rank", "children"),
    Output("lift-rank", "children"),
    Input("barchart", "hoverData")
)

def populate_report(hoverData):
    resortname = hoverData["points"][0]["customdata"][0]

    df_rank = resorts.query("Resort == @resortname")

    if df_rank.empty:
        return resortname, None, None, None, None

    else:
        try:
            priceRankCountry = df_rank['country_price_rank'].astype(int).iloc[0]
            elevationRankCountry = df_rank['country_elevation_rank'].astype(int).iloc[0]
            slopeRankCountry = df_rank['country_slope_rank'].astype(int).iloc[0]
            liftRankCountry = df_rank['country_lift_rank'].astype(int).iloc[0]

            priceRankGlobal = df_rank['global_price_rank'].astype(int).iloc[0]
            elevationRankGlobal = df_rank['global_elevation_rank'].astype(int).iloc[0]
            slopeRankGlobal = df_rank['global_slope_rank'].astype(int).iloc[0]
            liftRankGlobal = df_rank['global_lift_rank'].astype(int).iloc[0]

            priceRank = (f'Price: {priceRankCountry} ({priceRankGlobal})')
            elevationRank = (f'Elevation: {elevationRankCountry} ({elevationRankGlobal})')
            slopeRank = (f'Total slope: {slopeRankCountry} ({slopeRankGlobal})')
            liftRank = (f'Total lifts: {liftRankCountry} ({liftRankGlobal})')

        except IndexError as e:
            print(f"IndexError: {e}")

    return resortname, priceRank, elevationRank, slopeRank, liftRank

if __name__ == "__main__":
    app.run_server(port = 8051, debug = True, jupyter_mode="external")

Dash app running on http://127.0.0.1:8051/
